In [22]:
import pandas as pd

movies = pd.read_csv("movies_with_genre.csv")

In [23]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      top_k=None,
                      device=0)
classifier("I love this!")


Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119900392368436},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [25]:
classifier(movies['Overview'][0])

[[{'label': 'neutral', 'score': 0.7397616505622864},
  {'label': 'joy', 'score': 0.1546037197113037},
  {'label': 'disgust', 'score': 0.05240125581622124},
  {'label': 'sadness', 'score': 0.03464994952082634},
  {'label': 'anger', 'score': 0.014029540121555328},
  {'label': 'surprise', 'score': 0.0029920756351202726},
  {'label': 'fear', 'score': 0.0015617961762472987}]]

In [26]:
import numpy as np

def get_overview_emotion(sequence):
    prediction = classifier(sequence)
    scores = np.array([pred['score'] for pred in prediction[0]])
    index = np.argmax(scores)
    return prediction[0][index]['label']


get_overview_emotion(movies['Overview'][0])

'neutral'

In [27]:
from tqdm import tqdm

uni_id = []
emotional_tone = []

for i in tqdm(range(0, len(movies))):
    emotional_tone += [get_overview_emotion(movies['Overview'][i])]
    uni_id += [movies['Unique_id'][i]]
    
    
emotional_prediction_df = pd.DataFrame({"Unique_id": uni_id, "Emotional_tone": emotional_tone})  
emotional_prediction_df

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:36<00:00, 10.36it/s]


,Unique_id,Emotional_tone
0,1,neutral
1,2,disgust
2,3,fear
3,4,neutral
4,5,disgust
...,...,...
995,996,fear
996,997,disgust
997,998,disgust
998,999,disgust


In [28]:
movies = pd.merge(movies, emotional_prediction_df, on='Unique_id')

In [29]:
movies

,Poster_Link,Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Overview,Director,Star1,Star2,Star3,Star4,No_of_Votes,Unique_id,tagged_description,Simple_genre,Emotional_tone
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,1,1 Two imprisoned men bond over a number of yea...,Action,neutral
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,2,2 An organized crime dynasty's aging patriarch...,Crime,disgust
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,3,3 When the menace known as the Joker wreaks ha...,Crime,fear
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,4,4 The early life and career of Vito Corleone i...,Crime,neutral
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,5,5 A jury holdout attempts to prevent a miscarr...,Action,disgust
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,996,996 A young New York socialite becomes interes...,Action,fear
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,997,997 Sprawling epic covering the life of a Texa...,Drama,disgust
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,998,"998 In Hawaii in 1941, a private is cruelly pu...",Drama,disgust
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,999,999 Several survivors of a torpedoed merchant ...,War,disgust


In [30]:
movies.to_csv("movies_with_emotion.csv", index=False)